In [1]:
import os

import numpy as np
import torch
import accelerate

from rd3d.datasets import build_dataloader
from rd3d.core import Config
from rd3d import PROJECT_ROOT
from rd3d.models.dense_heads.point_segment_head import PointSegmentor

os.chdir(PROJECT_ROOT)
acc = accelerate.Accelerator()
cfg = Config.fromfile_py("configs/voxformer/voxformer_4x2_80e_kitti_3cls.py")
dataloader = build_dataloader(cfg.DATASET, cfg.RUN, training=True)
batch_dict = next(iter(dataloader))
dataloader.dataset.load_data_to_gpu(batch_dict)

[2023-12-17 16:48:41,033 cfg INFO] import module at root: /home/nrsl/workspace/temp/voxformer
[2023-12-17 16:48:41,034 cfg INFO] import module as config: configs.voxformer.voxformer_4x2_80e_kitti_3cls
[2023-12-17 16:48:41,180 dataset INFO] Database filter by min points Car: 14357 => 13532
[2023-12-17 16:48:41,181 dataset INFO] Database filter by min points Pedestrian: 2207 => 2168
[2023-12-17 16:48:41,181 dataset INFO] Database filter by min points Cyclist: 734 => 705
[2023-12-17 16:48:41,199 dataset INFO] Database filter by difficulty Car: 13532 => 10759
[2023-12-17 16:48:41,202 dataset INFO] Database filter by difficulty Pedestrian: 2168 => 2075
[2023-12-17 16:48:41,205 dataset INFO] Database filter by difficulty Cyclist: 705 => 581
[2023-12-17 16:48:41,212 dataset INFO] Loading KITTI dataset
[2023-12-17 16:48:41,312 dataset INFO] Total samples for KITTI dataset: 3712


In [2]:
from rd3d.models import build_detector
from tqdm import tqdm
model = build_detector(cfg.MODEL, dataset=dataloader.dataset).cuda()
model.eval()
for batch_dict in tqdm(iterable=dataloader):
    dataloader.dataset.load_data_to_gpu(batch_dict)
    batch_dict = model.vfe(batch_dict)
    vox_coors = batch_dict['voxel_features'][:, -1].int()
    assert (vox_coors != 0).all()

100%|██████████| 464/464 [02:05<00:00,  3.71it/s]
